In [7]:
spark

In [8]:
sc

<SparkContext master=spark://devenv:7077 appName=PySparkShell>

# 0. Import Data and Combine features

In [25]:
# Prepare data
raw_train = spark.read.csv("hdfs://devenv/user/spark/spark_mllib_101/train",
                      inferSchema=True,
                      header=True)
raw_test = spark.read.csv("hdfs://devenv/user/spark/spark_mllib_101/test",
                      inferSchema=True,
                      header=True)

In [26]:
raw_train.show(5)
raw_test.show(5)

+----------+-----------------+--------------------+---------------+-------------------+------------+------------+------------------+------------+------------+----------------+---------------+---------------+-----------------------------+----------------------+----------------------+---------------------------+-----+
|stock_code|stock_report_date|operating_gross_rate|net_profit_rate|revenue_growth_rate|current_rate|  quick_rate|cash_reinvest_rate|    roe_rate|    roa_rate|foreign_rate_bys|avg_import_rate|avg_export_rate|export_kgm_weight_37050000306|new_cases_smoothed_USA|new_cases_smoothed_TWN|new_cases_smoothed_OWID_EUR|label|
+----------+-----------------+--------------------+---------------+-------------------+------------+------------+------------------+------------+------------+----------------+---------------+---------------+-----------------------------+----------------------+----------------------+---------------------------+-----+
|      2302|            20211|        -0.10022

In [27]:
data_train = raw_train.select('label','operating_gross_rate',
       'net_profit_rate', 'revenue_growth_rate', 'current_rate', 'quick_rate',
       'cash_reinvest_rate', 'roa_rate', 'foreign_rate_bys',
       'avg_import_rate', 'avg_export_rate', 'export_kgm_weight_37050000306',
       'new_cases_smoothed_USA', 'new_cases_smoothed_TWN',
       'new_cases_smoothed_OWID_EUR',)
data_test = raw_test.select('label','operating_gross_rate',
       'net_profit_rate', 'revenue_growth_rate', 'current_rate', 'quick_rate',
       'cash_reinvest_rate', 'roa_rate', 'foreign_rate_bys',
       'avg_import_rate', 'avg_export_rate', 'export_kgm_weight_37050000306',
       'new_cases_smoothed_USA', 'new_cases_smoothed_TWN',
       'new_cases_smoothed_OWID_EUR',)

In [28]:
data_train.show(5)

+-----+--------------------+---------------+-------------------+------------+------------+------------------+------------+----------------+---------------+---------------+-----------------------------+----------------------+----------------------+---------------------------+
|label|operating_gross_rate|net_profit_rate|revenue_growth_rate|current_rate|  quick_rate|cash_reinvest_rate|    roa_rate|foreign_rate_bys|avg_import_rate|avg_export_rate|export_kgm_weight_37050000306|new_cases_smoothed_USA|new_cases_smoothed_TWN|new_cases_smoothed_OWID_EUR|
+-----+--------------------+---------------+-------------------+------------+------------+------------------+------------+----------------+---------------+---------------+-----------------------------+----------------------+----------------------+---------------------------+
|    0|        -0.100223498|   -0.007912025|         1.32096154|-0.481106719|-0.441839507|      -0.409650054|-0.460037026|    -0.605859958|   -1.511413276|   -1.511413276| 

In [29]:
data_test.show(5)

+-----+--------------------+---------------+-------------------+------------+------------+------------------+------------+----------------+---------------+---------------+-----------------------------+----------------------+----------------------+---------------------------+
|label|operating_gross_rate|net_profit_rate|revenue_growth_rate|current_rate|  quick_rate|cash_reinvest_rate|    roa_rate|foreign_rate_bys|avg_import_rate|avg_export_rate|export_kgm_weight_37050000306|new_cases_smoothed_USA|new_cases_smoothed_TWN|new_cases_smoothed_OWID_EUR|
+-----+--------------------+---------------+-------------------+------------+------------+------------------+------------+----------------+---------------+---------------+-----------------------------+----------------------+----------------------+---------------------------+
|    0|         0.175852981|    0.160705421|        0.711933009|-0.448060539|-0.408490147|      -0.319880882|-0.179077792|    -0.595941918|   -1.888952763|   -1.888952763| 

In [31]:
data_test.dtypes
data_train.dtypes

[('label', 'int'),
 ('operating_gross_rate', 'double'),
 ('net_profit_rate', 'double'),
 ('revenue_growth_rate', 'double'),
 ('current_rate', 'double'),
 ('quick_rate', 'double'),
 ('cash_reinvest_rate', 'double'),
 ('roa_rate', 'double'),
 ('foreign_rate_bys', 'double'),
 ('avg_import_rate', 'double'),
 ('avg_export_rate', 'double'),
 ('export_kgm_weight_37050000306', 'double'),
 ('new_cases_smoothed_USA', 'double'),
 ('new_cases_smoothed_TWN', 'double'),
 ('new_cases_smoothed_OWID_EUR', 'double')]

In [32]:
numCols = [x for (x, dataType) in data_train.dtypes if ((dataType == 'double')&(x != 'label'))]

In [33]:
print(numCols)

['operating_gross_rate', 'net_profit_rate', 'revenue_growth_rate', 'current_rate', 'quick_rate', 'cash_reinvest_rate', 'roa_rate', 'foreign_rate_bys', 'avg_import_rate', 'avg_export_rate', 'export_kgm_weight_37050000306', 'new_cases_smoothed_USA', 'new_cases_smoothed_TWN', 'new_cases_smoothed_OWID_EUR']


In [34]:
# Import the necessary class
from pyspark.ml.feature import VectorAssembler

# Create an assembler object
assembler = VectorAssembler(inputCols=numCols, outputCol='features')

# Consolidate predictor columns
data_assembled_train = assembler.transform(data_train)
data_assembled_test = assembler.transform(data_test)

# Check the resulting column
data_assembled_train.select('features', 'label').show(5, truncate=False)
data_assembled_test.select('features', 'label').show(5, truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                                                                                          |label|
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|[-0.100223498,-0.007912025,1.32096154,-0.481106719,-0.441839507,-0.409650054,-0.460037026,-0.605859958,-1.511413276,-1.511413276,2.823498971,1.928740454,-0.207683189,1.923804593]|0    |
|[-0.152181711,0.296171533,-0.027280983,-0.326541885,-0.28164175,0.116017502,0.248214375,1.280055378,-1.511413276,-1.511413276,2.823498971,1.928740454,-0.207683189,1.923804593]   |1    |
|[-0.650825464,0.040721151,-0.21208448,-0.539211174,-0.453028459,

In [36]:
# # Split into training and testing sets in a 80:20 ratio
# data_train, data_test = data_assembled.randomSplit([0.8, 0.2], seed=17)
# data_test 
# # Check that training set has around 80% of records
# training_ratio = data_train.count() / data.count()
# print(training_ratio)

data_train = data_assembled_train
data_test = data_assembled_test

# 1. DecisionTree

In [37]:
# Import the Decision Tree Classifier class
from pyspark.ml.classification import DecisionTreeClassifier

# Create a classifier object and fit to the training data
tree = DecisionTreeClassifier()
tree_model = tree.fit(data_train)

#label is y(ROE)
#features are all of X
# Create predictions for the testing data and take a look at the predictions
prediction = tree_model.transform(data_test)
prediction.select('label', 'prediction', 'probability').show(5, False)

+-----+----------+----------------------------------------+
|label|prediction|probability                             |
+-----+----------+----------------------------------------+
|0    |0.0       |[0.5922330097087378,0.4077669902912621] |
|1    |1.0       |[0.12115732368896925,0.8788426763110307]|
|1    |1.0       |[0.12115732368896925,0.8788426763110307]|
|1    |1.0       |[0.12115732368896925,0.8788426763110307]|
|1    |1.0       |[0.12115732368896925,0.8788426763110307]|
+-----+----------+----------------------------------------+
only showing top 5 rows



In [38]:
# Create a confusion matrix
prediction.groupBy('label', 'prediction').count().show()

# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()

# Accuracy measures the proportion of correct predictions
accuracy = (TN + TP) / (TN + TP + FN + FP)
print(accuracy)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|    6|
|    0|       0.0|   29|
|    1|       1.0|  103|
|    0|       1.0|   11|
+-----+----------+-----+

0.8859060402684564


# 2.Logistic Regression

In [39]:
# Import the logistic regression class
from pyspark.ml.classification import LogisticRegression

# Create a classifier object and train on training data
logistic = LogisticRegression().fit(data_train)

# Create predictions for the testing data and show confusion matrix
prediction = logistic.transform(data_test)
prediction.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       1.0|  109|
|    0|       1.0|   40|
+-----+----------+-----+



In [40]:
# Create a confusion matrix
prediction.groupBy('label', 'prediction').count().show()

# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()

# Accuracy measures the proportion of correct predictions
accuracy = (TN + TP) / (TN + TP + FN + FP)
print(accuracy)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       1.0|  109|
|    0|       1.0|   40|
+-----+----------+-----+

0.7315436241610739


In [73]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

# Calculate precision and recall
precision = TP / (TP + FP)
recall = TP / (TP + FN)
accuracy = (TN + TP) / (TN + TP + FN + FP)
print('precision = {:.2f}\nrecall    = {:.2f}'.format(precision, recall))

# Find weighted precision
multi_evaluator = MulticlassClassificationEvaluator()
weighted_precision = multi_evaluator.evaluate(prediction, {multi_evaluator.metricName: "weightedPrecision"})

# Find AUC
binary_evaluator = BinaryClassificationEvaluator()
auc = binary_evaluator.evaluate(prediction, {binary_evaluator.metricName: "areaUnderROC"})

precision = 0.82
recall    = 0.85


In [59]:
# Import the logistic regression class
from pyspark.ml.classification import LogisticRegression
accuracy_all = {}
F1_score_all = {}
lr_params = [0,0.001, 0.01, 0.1, 1, 10]

for r in lr_params:
    # Create a classifier object and train on training data
    logistic = LogisticRegression(regParam=r).fit(data_train)

    # Create predictions for the testing data and show confusion matrix
    prediction = logistic.transform(data_test)

    # Calculate the elements of the confusion matrix
    TN = prediction.filter('prediction = 0 AND label = prediction').count()
    TP = prediction.filter('prediction = 1 AND label = prediction').count()
    FN = prediction.filter('prediction = 0 AND label != prediction').count()
    FP = prediction.filter('prediction = 1 AND label != prediction').count()

    # Accuracy measures the proportion of correct predictions
    accuracy = (TN + TP) / (TN + TP + FN + FP)
    accuracy_all[f'regParam:{r}'] = accuracy
    try:
        Recall = TP/(TP+FN) #(召回率)
        Precision = TP/(TP+FP) #(準確率)
        F1_score = 2 * Precision * Recall / (Precision + Recall)
        F1_score_all[f'regParam:{r}_f1'] = F1_score
    except:
        pass

In [60]:
accuracy_all_sorted = dict(sorted(accuracy_all.items(), key=lambda item: item[1], reverse=True))
accuracy_all_sorted

{'regParam:0.1': 0.7449664429530202,
 'regParam:0.01': 0.738255033557047,
 'regParam:1': 0.738255033557047,
 'regParam:10': 0.738255033557047,
 'regParam:0': 0.7315436241610739,
 'regParam:0.001': 0.7315436241610739}

In [61]:
F1_score_all_sorted = dict(sorted(F1_score_all.items(), key=lambda item: item[1], reverse=True))
F1_score_all_sorted

{'regParam:0.1_f1': 0.8515625,
 'regParam:0.01_f1': 0.8482490272373542,
 'regParam:1_f1': 0.8482490272373542,
 'regParam:10_f1': 0.8482490272373542,
 'regParam:0_f1': 0.8449612403100776,
 'regParam:0.001_f1': 0.8449612403100776}

# 3. Random Forest

In [90]:
# Import the Decision Tree Classifier class
from pyspark.ml.classification import RandomForestClassifier

# Create a classifier object and fit to the training data
tree = RandomForestClassifier(maxDepth=10, numTrees=50)
tree_model = tree.fit(data_train)

#label is y(ROE)
#features are all of X
# Create predictions for the testing data and take a look at the predictions
prediction = tree_model.transform(data_test)
prediction.select('label', 'prediction', 'probability').show(5, False)

+-----+----------+-----------------------------------------+
|label|prediction|probability                              |
+-----+----------+-----------------------------------------+
|0    |1.0       |[0.3802570762636994,0.6197429237363005]  |
|1    |1.0       |[0.04794774700918855,0.9520522529908115] |
|1    |1.0       |[0.09829421721424084,0.9017057827857592] |
|1    |1.0       |[0.015705235426879845,0.9842947645731202]|
|1    |1.0       |[0.11858475860662772,0.8814152413933724] |
+-----+----------+-----------------------------------------+
only showing top 5 rows



In [91]:
# Create a confusion matrix
prediction.groupBy('label', 'prediction').count().show()

# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()

# Accuracy measures the proportion of correct predictions
accuracy = (TN + TP) / (TN + TP + FN + FP)
Recall = TP/(TP+FN) #(召回率)
Precision = TP/(TP+FP) #(準確率)
F1_score = 2 * Precision * Recall / (Precision + Recall)
print(accuracy)
print(F1_score)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|    6|
|    0|       0.0|   28|
|    1|       1.0|  103|
|    0|       1.0|   12|
+-----+----------+-----+

0.8791946308724832
0.9196428571428571


In [92]:
prediction_prediction = prediction.select("prediction").rdd.flatMap(lambda x: x).collect()
prediction_prediction

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0]

In [93]:
prediction_label = prediction.select("label").rdd.flatMap(lambda x: x).collect()
prediction_label

[0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1]

In [52]:
# Import the Decision Tree Classifier class
from pyspark.ml.classification import RandomForestClassifier
accuracy_all = {}
F1_score_all = {}
Depth = [0, 1, 3, 5, 7, 10]
tree_num = [10, 50, 100, 150, 200]
# Create a classifier object and fit to the training data
for d in Depth:
    for t in tree_num:
        tree = RandomForestClassifier(maxDepth=d, numTrees=t)
        tree_model = tree.fit(data_train)

        #label is y(ROE)
        #features are all of X
        # Create predictions for the testing data and take a look at the predictions
        prediction = tree_model.transform(data_test)
#         prediction.select('label', 'prediction', 'probability').show(5, False)
        # Create a confusion matrix
#         prediction.groupBy('label', 'prediction').count().show()

        # Calculate the elements of the confusion matrix
        TN = prediction.filter('prediction = 0 AND label = prediction').count()
        TP = prediction.filter('prediction = 1 AND label = prediction').count()
        FN = prediction.filter('prediction = 0 AND label != prediction').count()
        FP = prediction.filter('prediction = 1 AND label != prediction').count()

        # Accuracy measures the proportion of correct predictions
        accuracy = (TN + TP) / (TN + TP + FN + FP)
        accuracy_all[f'Tree:{t} & Depth:{d}'] = accuracy
        try:
            Recall = TP/(TP+FN) #(召回率)
            Precision = TP/(TP+FP) #(準確率)
            F1_score = 2 * Precision * Recall / (Precision + Recall)
            F1_score_all[f'Tree:{t} & Depth:{d}_f1'] = F1_score
        except:
            pass

In [54]:
accuracy_all_sorted = dict(sorted(accuracy_all.items(), key=lambda item: item[1], reverse=True))
F1_score_all_sorted = dict(sorted(F1_score_all.items(), key=lambda item: item[1], reverse=True))
print(accuracy_all_sorted)


{'Tree:150 & Depth:3': 0.8791946308724832, 'Tree:50 & Depth:10': 0.8791946308724832, 'Tree:150 & Depth:1': 0.87248322147651, 'Tree:50 & Depth:3': 0.87248322147651, 'Tree:200 & Depth:3': 0.87248322147651, 'Tree:100 & Depth:5': 0.87248322147651, 'Tree:150 & Depth:5': 0.87248322147651, 'Tree:200 & Depth:5': 0.87248322147651, 'Tree:10 & Depth:7': 0.87248322147651, 'Tree:200 & Depth:7': 0.87248322147651, 'Tree:100 & Depth:10': 0.87248322147651, 'Tree:50 & Depth:1': 0.8657718120805369, 'Tree:200 & Depth:1': 0.8657718120805369, 'Tree:10 & Depth:5': 0.8657718120805369, 'Tree:100 & Depth:7': 0.8657718120805369, 'Tree:200 & Depth:10': 0.8657718120805369, 'Tree:100 & Depth:3': 0.8590604026845637, 'Tree:50 & Depth:5': 0.8590604026845637, 'Tree:50 & Depth:7': 0.8590604026845637, 'Tree:150 & Depth:7': 0.8590604026845637, 'Tree:150 & Depth:10': 0.8590604026845637, 'Tree:10 & Depth:1': 0.8523489932885906, 'Tree:100 & Depth:1': 0.8523489932885906, 'Tree:10 & Depth:3': 0.8523489932885906, 'Tree:10 & Dep

In [55]:
print(F1_score_all_sorted)

{'Tree:50 & Depth:10_f1': 0.9196428571428571, 'Tree:150 & Depth:3_f1': 0.918918918918919, 'Tree:100 & Depth:5_f1': 0.9155555555555556, 'Tree:150 & Depth:5_f1': 0.9155555555555556, 'Tree:200 & Depth:5_f1': 0.9155555555555556, 'Tree:200 & Depth:7_f1': 0.9155555555555556, 'Tree:100 & Depth:10_f1': 0.9155555555555556, 'Tree:10 & Depth:7_f1': 0.9147982062780269, 'Tree:150 & Depth:1_f1': 0.9140271493212669, 'Tree:50 & Depth:3_f1': 0.9140271493212669, 'Tree:200 & Depth:3_f1': 0.9140271493212669, 'Tree:100 & Depth:7_f1': 0.911504424778761, 'Tree:200 & Depth:10_f1': 0.911504424778761, 'Tree:50 & Depth:1_f1': 0.9090909090909092, 'Tree:200 & Depth:1_f1': 0.9090909090909092, 'Tree:10 & Depth:5_f1': 0.9090909090909092, 'Tree:150 & Depth:7_f1': 0.907488986784141, 'Tree:150 & Depth:10_f1': 0.907488986784141, 'Tree:50 & Depth:7_f1': 0.9066666666666666, 'Tree:50 & Depth:5_f1': 0.9058295964125561, 'Tree:100 & Depth:3_f1': 0.903225806451613, 'Tree:10 & Depth:10_f1': 0.9026548672566371, 'Tree:10 & Depth:1

In [18]:
# Create a confusion matrix
prediction.groupBy('label', 'prediction').count().show()

# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()

# Accuracy measures the proportion of correct predictions
accuracy = (TN + TP) / (TN + TP + FN + FP)
print(accuracy)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|   24|
|    0|       0.0|  128|
|    1|       1.0|  173|
|    0|       1.0|   26|
+-----+----------+-----+

0.8575498575498576


In [24]:
accuracy_all = {}

Depth = [0, 1, 3, 5, 7, 10]
tree = [10, 50, 100, 150, 200]
t = 50
a =50
d =6
accuracy_all[f'Tree:{t} & Depth:{d}'] = a
accuracy_all

{'Tree:50 & Depth:6': 50}